In [115]:
import numpy as np
import tk as tk

In [116]:
class Sudoku:
    def __init__(self):
        self.sudoku     = self._initialize_sudoku()
        self.numberlist = self._initialize_numberlist()
        self.positions  = self._initialize_positions()

    @staticmethod
    def _initialize_sudoku():
        return np.zeros((9,9),dtype=int)

    @staticmethod
    def _initialize_positions():
        return [(i,j) for i in range(0,9) for j in range(0,9)]

    @staticmethod
    def _initialize_numberlist():
        return np.array([1,2,3,4,5,6,7,8,9])

    def _is_possible(self,row,col,number):
        ## Checking column
        for i in range(0,9):
            if self.sudoku[i][col] == number:
                return False
        ## Checking row
        for i in range(0,9):
            if self.sudoku[row][i] == number:
                return False
        ## Checking square
        row_0, col_0 = (row//3) * 3, (col//3) * 3
        for i in range(0,3):
            for j in range(0,3):
                if self.sudoku[row_0 + i][col_0 + j] == number:
                    return False
        return True

    def _check_sudoku(self):
        for row in range(0 ,9):
            for col in range(0 ,9):
                if self.sudoku[row][col] == 0:
                    return False
        # We have a complete grid!
        return True

    def _permute_numberlist(self):
        self.numberlist = np.random.permutation(self.numberlist)

    def _permute_positions(self):
        self.positions = np.random.permutation(self.positions)

    def fill_sudoku(self):
        self._permute_numberlist()
        # Find next empty cell
        row, col = None, None
        for i in range(0, 81):
            row = i // 9
            col = i % 9
            if self.sudoku[row][col] == 0:
                for number in self.numberlist:
                    # Check that this value has not already be used
                   if self._is_possible(row,col,number):
                        self.sudoku[row][col] = number
                        if self._check_sudoku():
                            return True
                        else:
                            if self.fill_sudoku():
                                return True
                break
        self.sudoku[row][col] = 0

    def generate_unique_sudoku(self):
        currently_removed = None
        self._permute_positions()
        pos_nr = 0
        while pos_nr < 81:
            row,col               = self.positions[pos_nr]
            currently_removed     = self.sudoku[row][col]
            self.sudoku[row][col] = 0
            self._permute_numberlist()
            solution_counter = 0
            for number in self.numberlist:
                if self._is_possible(row,col,number):
                    solution_counter += 1
            if solution_counter > 1:
                self.sudoku[row][col] = currently_removed
                pos_nr += 1
            elif solution_counter == 1:
                pos_nr += 1
            elif solution_counter == 0:
                break

    def print_sudoku(self):
        for row in range(self.sudoku.shape[0]):
            if row % 3 == 0 and row != 0:
                print("- - - - - - - - - - - - ")
            for col in range(self.sudoku.shape[1]):
                if col % 3 == 0 and col != 0:
                    print(" | ", end = "")
                if col == 8:
                    print(self.sudoku[row][col])
                else:
                    print(str(self.sudoku[row][col]) + " ", end="")




In [117]:
my_sudoku = Sudoku()
my_sudoku.fill_sudoku()
my_sudoku.print_sudoku()

7 1 9  | 4 2 8  | 5 3 6
3 6 4  | 7 5 1  | 8 9 2
8 5 2  | 3 9 6  | 1 4 7
- - - - - - - - - - - - 
9 7 5  | 1 4 3  | 6 2 8
2 4 1  | 8 6 9  | 3 7 5
6 8 3  | 2 7 5  | 4 1 9
- - - - - - - - - - - - 
1 9 8  | 6 3 7  | 2 5 4
4 3 7  | 5 8 2  | 9 6 1
5 2 6  | 9 1 4  | 7 8 3


In [118]:
my_sudoku.generate_unique_sudoku()
my_sudoku.print_sudoku()

0 0 9  | 0 2 0  | 5 3 0
0 6 4  | 7 0 0  | 0 9 2
0 0 0  | 3 9 0  | 0 0 7
- - - - - - - - - - - - 
0 0 5  | 0 4 3  | 6 2 8
2 4 0  | 8 6 9  | 0 0 0
6 8 0  | 0 0 5  | 4 1 0
- - - - - - - - - - - - 
1 0 0  | 6 0 7  | 0 5 0
0 3 0  | 5 8 0  | 9 6 0
5 2 0  | 9 0 0  | 0 0 3
